# 1. Import the relevant libraries, along with your ords_prods_merge dataframe.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# define the pathname

path = r'/Users/iMac/Library/CloudStorage/OneDrive-Personal/Data Analytics/Data Analytics Immersion/Exercise 4/2024-01_Instacart Basket Analysis'

In [3]:
# import ords_prods_merge DataFrame

df = pd.read_pickle(os.path.join(path, '02 Data', '00 Prepared Data', 'orders_products_derivations.pkl'))

# 2. Find the aggregated mean of the “order_number” column grouped by “department_id” for the entire dataframe.

In [4]:
df.groupby('department_id').agg({'order_number': ['mean']})

order_number
                      mean
department_id             
1                15.457838
2                17.277920
3                17.170395
4                17.811403
5                15.215751
6                16.439806
7                17.225802
8                15.340650
9                15.895474
10               20.197148
11               16.170638
12               15.887671
13               16.583536
14               16.773669
15               16.165037
16               17.665606
17               15.694469
18               19.310397
19               17.177343
20               16.473447
21               22.902379

# 3. Analyze the result. How do the results for the entire dataframe differ from those of the subset?

The mean order number for each department in the entire dataframe is slightly different from the mean order number in the subset, which is not surprising, given the limited nature of the subset.

For example, comparing department_id 1: 

    Entire DataFrame: 15.457838
    Subset: 15.577493

Similarly, the mean order numbers for other departments also show some differences between the entire dataframe and the subset, but they are not significantly at variance with one another.

Departments with higher mean values in the subset may not necessarily have the same ranking in the entire dataframe, as the subset might not capture the full diversity of the data.

# 4. Create a loyalty flag for existing customers using the transform() and loc() functions

In [5]:
# create 'max_order' column

df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)

/var/folders/6c/bwpb08p57j1fmq58t70xfnf80000gq/T/ipykernel_6812/1599324200.py:3: FutureWarning: The provided callable <function amax at 0x107cfc400> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)


## @Kimera: when this warning came up, I did some research and found the alternative below; hope that's acceptable

In [6]:
# create 'max_order' column

df['max_order'] = df.groupby(['user_id'])['order_number'].transform('max')

# create 'loyaly_flag' based on the corresponding 'max_order' value.

df.loc[df['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df.loc[(df['max_order'] <= 40) & (df['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df.loc[df['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [7]:
df[['user_id', 'order_number', 'loyalty_flag', 'max_order']].head(10)

user_id  order_number      loyalty_flag  max_order
0      138            28  Regular customer         32
1      138            30  Regular customer         32
2      709             2      New customer          5
3      764             1      New customer          3
4      764             3      New customer          3
5      777            16  Regular customer         26
6      825             3      New customer          9
7      910            12  Regular customer         12
8     1052            10  Regular customer         20
9     1052            15  Regular customer         20

# 5. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer).

In [8]:
# Basic statistics of product prices for each loyalty category

df.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

### It seems that on average, loyal customers tend to purchase products at a slightly lower mean price compared to new and regular customers. However, the high maximum prices suggest the presence of outliers in all loyalty categories.

# 6. Create a spending flag for each user based on the average price across all their orders.

In [9]:
# Create 'spending_flag' column based on the mean spending value

df['spender_type'] = df.groupby('user_id')['prices'].transform('mean')

# Assign labels based on the spending_flag values

df.loc[df['spender_type'] < 10, 'spending_flag'] = 'Low spender'
df.loc[df['spender_type'] >= 10, 'spending_flag'] = 'High spender'

In [10]:
df[['user_id', 'spending_flag']].head(10)

user_id spending_flag
0      138   Low spender
1      138   Low spender
2      709   Low spender
3      764   Low spender
4      764   Low spender
5      777   Low spender
6      825   Low spender
7      910   Low spender
8     1052   Low spender
9     1052   Low spender

# 7. Create an order frequency flag that marks the regularity of a user’s ordering behavior.

In [11]:
# Calculate the median of "days_since_prior_order" for each user

df['order_freq_flag'] = df.groupby('user_id')['days_since_prior_order'].transform('median')

# Create the order frequency flag based on the median

df.loc[df['order_freq_flag'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
df.loc[(df['order_freq_flag'] > 10) & (df['order_freq_flag'] <= 20), 'order_frequency_flag'] = 'Regular customer'
df.loc[df['order_freq_flag'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [12]:
df[['user_id', 'order_freq_flag']].head(10)

user_id  order_freq_flag
0      138              8.0
1      138              8.0
2      709              8.0
3      764              9.0
4      764              9.0
5      777             11.0
6      825             20.0
7      910              6.0
8     1052             10.0
9     1052             10.0

# 9. Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [13]:
# Export data to pkl

df.to_pickle(os.path.join(path, '02 Data','00 Prepared Data', 'customers.pkl'))